![](../../images/rivacon_frontmark_combined_header.png)

# Day Counter, Roll Conventions and Schedules

In [1]:
import os, sys, holidays
from datetime import date, timedelta, datetime

# Get the current working directory
current_dir = os.getcwd()

# Get the grandparent directory
grandparent_dir = os.path.abspath(os.path.join(current_dir, "..", ".."))

# Add the grandparent directory to sys.path
if grandparent_dir not in sys.path:
    sys.path.insert(0, grandparent_dir)

print("Grandparent directory containing Rivapy module added to sys.path:", grandparent_dir)
from rivapy.tools.enums import RollConvention
from rivapy.tools.datetools import Period, Schedule, roll_day

Grandparent directory containing Rivapy module added to sys.path: C:\Users\Anwender\Desktop\RiVaPy


C:\Users\Anwender\.conda\envs\python_3.11\Lib\site-packages\pyvacon\__init__.py:12: UserWarning: Imported pyvacon is just beta version.
  _warnings.warn('Imported pyvacon is just beta version.')
C:\Users\Anwender\Desktop\RiVaPy\rivapy\__init__.py:9: UserWarning: Imported pyvacon is just beta version.
  warnings.warn('Imported pyvacon is just beta version.')


## Schedules
For different financial instruments such as Bonds or Swaps a vector for certain dates is needed. These vectors are usually called schedules. They may describe the dates where certain payments or other events (such as fixings) occur. Such schedules are normally based on some construction logic such as "the first Monday of each month in year 2020". The generation of these schedules is therefore often not handmade but made by algorithms, which we call schedule generators. To create such a schedule with a generator, we have three main ingredients:
- Periods describing the frequency of the dates such as monthly, yearly, quarterly etc.
- A holiday calendar since most often schedules contain only business days
- A roll convention which defines what happens if the algorithm ends at a holiday
which we discuss in the following.

### Periods
A period is described by the number of years/months/days and is the basis in the schedule generation.

In [2]:
period_1yr = Period(1,0,0) # create a period of 1 year (first argument of this method describes years, second month and last days)
period_3m = Period(0,3,0)
period_30days = Period(0,0,30)

### Holiday calendar
We use the Python library [holidays](https://holidays.readthedocs.io/en/latest/) as a Holiday calendar for determining business days.
See example using the holiday calendar of the Euopean Central Bank below.

In [3]:
ecb_holidays = holidays.ECB(2024)       # Optionally a year or a list of years can be provided. 
                                        # By default the years are automatically extended when needed, see below.

for d,n in ecb_holidays.items():
    print(d,n)
    
print(ecb_holidays.is_working_day("2023-1-1"))  # This can be used to check for working day (holidays and weekends considered) 
                                                # and adds 2023 to ecb_holidays

for d,n in ecb_holidays.items():
    print(d,n)

2024-01-01 New Year's Day
2024-03-29 Good Friday
2024-04-01 Easter Monday
2024-05-01 1 May (Labour Day)
2024-12-25 Christmas Day
2024-12-26 26 December
False
2024-01-01 New Year's Day
2024-03-29 Good Friday
2024-04-01 Easter Monday
2024-05-01 1 May (Labour Day)
2024-12-25 Christmas Day
2024-12-26 26 December
2023-01-01 New Year's Day
2023-04-07 Good Friday
2023-04-10 Easter Monday
2023-05-01 1 May (Labour Day)
2023-12-25 Christmas Day
2023-12-26 26 December


### Rolling Conventions

Rolling conventions, also known as business day adjustment rules, determine how dates are adjusted if they fall on non-business days (e.g., weekends or holidays). These conventions are essential in determining the exact timing of payments or other financial events. Common rolling conventions include:

1. **Unadjusted**  
   The date is not adjusted, even if it falls on a non-business day.
      
2. **Following**  
   If the date falls on a non-business day, it is rolled forward to the next business day.

3. **Modified Following**  
   Similar to Following, but if rolling forward moves the date to the next month, it is rolled backward to the preceding business day.

4. **Preceding**  
   If the date falls on a non-business day, it is rolled backward to the previous business day.

5. **Modified Preceding**  
   Similar to Preceding, but if rolling backward moves the date to the previous month, it is rolled forward to the next business day.


In [4]:
# Example of selecting a Roll Convention in rivapy
roll_conventions = [
                    RollConvention.UNADJUSTED,
                    RollConvention.FOLLOWING,
                    RollConvention.MODIFIED_FOLLOWING,
                    RollConvention.PRECEDING,
                    RollConvention.MODIFIED_PRECEDING,
                    ]

In [5]:
# 01.01.2021 is a friday, notice how MODIFIED_PRECEDING is the following business day
for roll_convention in roll_conventions:
    rolled_day = roll_day(day = date(2021,1,1),
                         calendar = holidays.ECB(),
                         business_day_convention = roll_convention,
                         start_day = None)
    print(rolled_day.date(), "\t ", roll_convention.name)


2021-01-01 	  UNADJUSTED
2021-01-04 	  FOLLOWING
2021-01-04 	  MODIFIED_FOLLOWING
2020-12-31 	  PRECEDING
2021-01-04 	  MODIFIED_PRECEDING


### Schedule generation
To create a schedule we first have to create a specification containing the information described above (roll conventions, holidays, periods). We may then call the generate method to create a list of dates defining the schedule.

**Additional Options:**

- **`backwards`** *(bool, optional)*:  
  Defines the direction for rolling out the schedule.  
  - **True**: The schedule will be rolled out *backwards* (from the end day to the start day).
  - **False**: The schedule will be rolled out *forwards* (from the start day to the end day). 
  - Defaults to **True**.

- **`stub`** *(bool, optional)*:  
  Defines the behavior for the first/last period when it is shorter than the others.  
  - **True**: The first/last period is accepted even if it is shorter.  
  - **False**: The remaining days of the shorter period are added to the neighboring period.  
  - Defaults to **True**.

In [6]:
#no business day adjustments
unadjusted_schedule = Schedule._roll_out(from_=date(2024,1,1), to_=date(2025,1,1), term=period_3m, backwards=False,
                  allow_stub=True)
# Show undajusted schedule
print("Unadjusted:")
for s in unadjusted_schedule: 
    print(s.date())
print()

# adjust for business days (holidays and weekends)
schedule_2024_3m_period = Schedule(start_day = date(2024,1,1),
                                    end_day = date(2025,1,1),
                                    time_period = period_3m,
                                    backwards = False,
                                    stub = False,
                                    business_day_convention = RollConvention.MODIFIED_FOLLOWING,
                                    calendar = holidays.ECB())
adjusted_schedule = schedule_2024_3m_period.generate_dates(ends_only=False)
print("Adjusted for Working Day:")
for s in adjusted_schedule: 
    print(s.date())


Unadjusted:
2024-01-01
2024-04-01
2024-07-01
2024-10-01
2025-01-01

Adjusted for Working Day:
2024-01-02
2024-04-02
2024-07-01
2024-10-01
2025-01-02


### Day Count Conventions
Let's imagine we have a financial product (e.g. Bond) which pays us interest of 5% p.a. each month over a year. Additionally let's ignore business days and assume we receive a coupon every month with the schedule being 01.01.2025, 01.02.2025, ..., 01.01.2026. Without further information, it is not clear how the coupon (as a year fraction of 5% times Nominal) is calculated for each monthly period:
- Should the the coupon for the first period in January (01.01.2025 - 01.02.2025, 31 days) be the same as for the second period in February (01.02.2025 - 01.03.2025, 28 days)?
- Are the period start days e.g. 01.01.2025 or the period end days e.g. 01.01.2026 included or excluded in the calculation?
  
**Day Count Conventions** (or Date Conventions, Day Count Fractions) give answers to these questions by defining how the time between two dates is measured for financial calculations. They determine how interest accrues between dates. An official source on day count conventions as well as other market conventions can be found [here](https://web.archive.org/web/20140913145444/http://www.hsbcnet.com/gbm/attachments/standalone/2006-isda-definitions.pdf). For an overview you can check out [this](https://strata.opengamma.io/day_counts/).


#### Actual/Actual (ISDA)

This Day Count Convention counts the actual days between period start and period end date divided by the actual number of days in the year (365 or 366 in a leap year). If the period spans over a leap year and a normal year, the days are splitted and the year fraction is computed as: 

$$ \frac{Days\; in\; non leap\; year}{365} + \frac{Days\; in\; leap\; year}{366}$$

The **first day** in the period is **included**, the **last day is excluded**.

Actual/Actual (ISDA) is primarily used in derivatives markets and swaps.

In [7]:
from rivapy.tools.datetools import DayCounter as dc
# Here you see an example using rivapy
# The period contains 31 days in the leap year 2024 and 31 days in the nonleap year 2025
print(dc("ActAct").yf(date(2024,12,1), date(2025,2,1)))   #last day is excluded
print(31/366+31/365) # Should be the same result

0.16963096040122763
0.16963096040122763


#### Actual/365 (Fixed)

This can be seen as a simplification of the Actual/Actual (ISDA) Convention, where the denominator is always 365.

#### Actual/360

Same as the Convention above, but the denominator is always 360.




In [8]:
d1 = date(2024,1,1)
d2 = date(2025,1,1)
print("Days in period:",(d2 - d1).days)

print("Act365Fixed:\t",dc("Act365Fixed").yf(d1, d2)) # equal to 366/365
print("Act360:\t\t",dc("Act360").yf(d1, d2))         # equal to 366/360

Days in period: 366
Act365Fixed:	 1.0027397260273974
Act360:		 1.0166666666666666


#### ACT/ACT (ICMA)

This Day Count Convention is commonly used in bond markets to calculate the day count fraction, particularly for determining accrued interest and bond yields. It provides a standardized way to calculate the fraction of a year represented by a given period, based on actual calendar days and the bond's coupon schedule.

$$
\text{Day Count Fraction} = \frac{d_2-d_1}{\text{Days in Coupon Period} \times \text{Number of Coupon Periods in a Year}}
$$

Where:
- **$d_2-d_1$**: is the number of dates between the two dates.
- **Days in Coupon Period**: The length of the current coupon period in days.
- **Number of Coupon Periods in a Year**: Based on the bond's coupon payment frequency (e.g., 1 for annual, 2 for semi-annual, 4 for quarterly).

The **first day** in the period is **included**, the **last day is excluded**.

**Example**:
Semi-annual bond with last coupon on
1st May, next coupon on 1 November (number of days: 184). On 31st May, the year franction is calculated as:
$$\frac{30}{184 \times 2} = \frac{30}{368}$$

In [9]:
# ACT/ACT ICMA is used for bonds and requires additional arguments for coupon period and frequency 
print(dc.yf_ActActICMA(date(2024,5,1), date(2024,5,31), coupon_period_in_days=184, coupon_frequency=2))
print(30/368) # Should be the same result

0.08152173913043478
0.08152173913043478


#### 30/360 (ISDA) or "Bond Basis"

This convention assumes that each month has 30 days and that the year has 360 days. 
The formula is therefore given by:

$$ \frac{360 (Y_{2} - Y_{1}) + 30(M_{2}-M_{1}) + D_{2}-D_{1}}{360}$$ 

where 
- $Y_{1}$ is the year of the period start date.
- $Y_{2}$ is the year of the period end date (day immeadiately following last day included).
- $M_{1}$ is the month (expressed as a number) of the period start date.
- $M_{2}$ is the month (expressed as a number) of the period end date (day immeadiately following last day included).
- $D_{1}$ is the calendar day (expressed as a number) of the period start date, unless this number is 31, in which case $D_{1}$ is set to 30.
- $D_{2}$ is the calendar day (expressed as a number) of the period end date (day immeadiately following last day included), unless this number is 31 and $D_{1}>29$, in which case $D_{2}$ is set to 30.
  


In [10]:
# in this example we have 31 included days, because the period end date is excluded
print(dc.yf_30360ISDA(date(2025,1,1), date(2025,2,1)))
# Y1=Y2=2025, M1=1, M2=2, D1=D2=1 
# the result should be 30/360=1/12=0,083333...

# now we move the period one day back (still 31 included days)
print(dc.yf_30360ISDA(date(2024,12,31), date(2025,1,31)))
# Y1=2024, Y2=2025, M1=12, M2=1, D1=D2=30 
# the result should be (360+30*(-11))/360=1/12=0,083333...
# and same as picking one or both of the calendar days to 30th e.g. date(2024,12,30) and date(2025,1,31)

# we pick a period where D1 and D2 is not adjusted (still 31 included days)
print(dc.yf_30360ISDA(date(2025,4,29), date(2025,5,30)))
# Y1=Y2=2025, M1=4, M2=5, D1=29, D2=30 
# the result should be (30+1)/360=0,08611111...

# moving the period one day forward changes D1 but not D2 (still 31 included days)
print(dc.yf_30360ISDA(date(2025,4,30), date(2025,5,31)))
# Y1=Y2=2025, M1=4, M2=5, D1=30, D2=30 (because D1>29)
# the result should be 30/360=1/12=0,083333...

0.08333333333333333
0.08333333333333337
0.08611111111111111
0.08333333333333333


#### 30E/360 or "Eurobond Basis"

This convention follows the same formula as the previous 30/360 (ISDA) convention, but has a simpler adjustment to $D_{2}$:

$$ \frac{360 (Y_{2} - Y_{1}) + 30(M_{2}-M_{1}) + D_{2}-D_{1}}{360}$$ 

where 
- $Y_{1}$ is the year of the period start date.
- $Y_{2}$ is the year of the period end date (day immeadiately following last day included).
- $M_{1}$ is the month (expressed as a number) of the period start date.
- $M_{2}$ is the month (expressed as a number) of the period end date (day immeadiately following last day included).
- $D_{1}$ is the calendar day (expressed as a number) of the period start date, unless this number is 31, in which case $D_{1}$ is set to 30.
- $D_{2}$ is the calendar day (expressed as a number) of the period end date (day immeadiately following last day included), **unless this number is 31, in which case $D_{2}$ is set to 30**.
  


In [11]:
# all of the examples below have the same result
print(dc.yf_30E360(date(2024,12,31), date(2025,1,31)))
print(dc.yf_30E360(date(2024,12,31), date(2025,1,30)))
print(dc.yf_30E360(date(2024,12,30), date(2025,1,31)))
print(dc.yf_30E360(date(2024,12,30), date(2025,1,30)))
# Y1=2024, Y2=2025, M1=12, M2=1, D1=D2=30 
# the result should be (360+30*(-11))/360=1/12=0,083333...

0.08333333333333337
0.08333333333333337
0.08333333333333337
0.08333333333333337


#### 30U/360 or 30/360 US

This convention is also known as the 30/360 U.S. Treasury method and is an extension the 30/360 (ISDA) convention. The only difference is the handling of the last day in february:

$$ \frac{360 (Y_{2} - Y_{1}) + 30(M_{2}-M_{1}) + D_{2}-D_{1}}{360}$$ 

where 
- $Y_{1}$ is the year of the period start date.
- $Y_{2}$ is the year of the period end date (day immeadiately following last day included).
- $M_{1}$ is the month (expressed as a number) of the period start date.
- $M_{2}$ is the month (expressed as a number) of the period end date (day immeadiately following last day included).
- $D_{1}$ is the calendar day (expressed as a number) of the period start date, unless:
    - if $D_{1}=31$, set $D_{1}=30$.
    - **if $D_{1}$ is the last day of february** ($M_{1}=2$, $D_{1}=29$ in leap years or $D_{1}=28$ in nonleap years), **set $D_{1}=30$**
- $D_{2}$ is the calendar day (expressed as a number) of the period end date (day immeadiately following last day included), unless:
    -  if $D_{2}=31$ and $D_{1}>29$, set $D_{2}=30$.
    -  **if $D_{1}$ and $D_{2}$ are the last days of february, set $D_{2}=30$**.
  

In [12]:
print(dc.yf_30U360(date(2024,2,29), date(2025,2,28)))
#Y1=2024, Y2=2025, M1=M2=2, D1=D2=30

print(dc.yf_30U360(date(2024,2,28), date(2025,2,28)))
#Y1=2024, Y2=2025, M1=M2=2, D1=D2=28

print(dc.yf_30U360(date(2023,2,28), date(2024,2,28))) # (360+28-30)/360
#Y1=2023, Y2=2024, M1=M2=2, D1=30, D2=28

1.0
1.0
0.9944444444444445


Below you can find a function printing all the introduced Day Count Conventions at once for a specified period:

In [13]:
from rivapy.tools.datetools import DayCounter as dc

# this is how you can get the year fraction based on a date count convention in rivapy:
# Options:
#   - "Act365Fixed"
#   - "ActAct"
#   - "Act360"
#   - "30U360"
#   - "30E360"
#   - "30360ISDA"
d1 = date(2024,1,1)
d2 = date(2024,7,1)

dc("ActAct").yf(d1, d2)

#printing all implemented types in rivapy

def show_day_counter(d1,d2,dc_names):
    if isinstance(dc_names, str):
        dc_names = [dc_names]
    print("d1"+":", d1)
    print("d2"+":", d2)
    for dc_name in dc_names:
        print(dc_name + ":",dc(dc_name).yf(d1,d2))
        
show_day_counter(date(2024,1,1),date(2024,7,1),["30360ISDA","30E360", "30U360","ActAct", "Act360","Act365Fixed"])

d1: 2024-01-01
d2: 2024-07-01
30360ISDA: 0.5
30E360: 0.5
30U360: 0.5
ActAct: 0.4972677595628415
Act360: 0.5055555555555555
Act365Fixed: 0.4986301369863014
